In [ ]:
import gym_kuka_multi_blocks.envs.kuka_multi_blocks_gym_env as e

In [ ]:
import numpy as np

In [ ]:
env = e.KukaMultiBlocksEnv(renders=True, numObjects=3, removeHeightHack=True)

# Random policy

In [ ]:
env.reset()

H = 1000
obs_R = np.zeros(H)
for k in range(H):
    obs, rew, done, info = env.step([0,0,-1,0])#env.action_space.sample())
    obs_R[k] = rew
    if done: 
        # Note that behaviour of environments is not specified when one tries
        # to step() on terminal states.
        break 
    
print('Reward: {}'.format(np.sum(obs_R)))

# PPDG

In [1]:
import ray
from ray.tune.logger import pretty_print
from ray.tune.registry import register_env
from ray.rllib.agents import ddpg

In [2]:
def env_creator(renders=False):
    import gym
    import gym_kuka_multi_blocks.envs.kuka_multi_blocks_gym_env as e
    env = e.KukaMultiBlocksEnv(renders=renders,
                               numObjects=3,
                               removeHeightHack=True,
                               isDiscrete=False)
    return env

## PPDG train

In [ ]:
register_env("my_env", env_creator)

config = ddpg.DEFAULT_CONFIG.copy()
config["num_workers"] = 4
config["horizon"] = 1000

ray.init()

agent = ddpg.DDPGAgent(config=config, env="my_env")
#agent.restore("/Users/dgrebenyuk/ray_results/2018-09-30_15-37-54bt93cdbw/checkpoint-842")

for i in range(1001):
    # Perform one iteration of training the policy with PPO
    result = agent.train()
    print(pretty_print(result))

    if i % 20 == 0:
        checkpoint = agent.save()
        print("checkpoint saved at", checkpoint)

## PPDG test

In [3]:
register_env("my_env", env_creator)

config = ddpg.DEFAULT_CONFIG.copy()
config["num_workers"] = 4 # <--- change here
config["horizon"] = 1000

ray.init()

env = env_creator(renders=True)

agent = ddpg.DDPGAgent(config=config, env="my_env")
agent.restore("./trained_policies/KukaMultiBlocks-v0-DDPG/2018-09-30_15-37-54bt93cdbw/checkpoint-842")

obs = env.reset()

reward = 0
for _ in range(1000):
    action_ = agent.compute_action(obs)
    obs, rew, done, _ = env.step(action_)
    reward += rew
print(reward)

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:59683 to respond...
Waiting for redis server at 127.0.0.1:54057 to respond...
Starting local scheduler with the following resources: {'CPU': 4, 'GPU': 0}.

View the web UI at http://localhost:8889/notebooks/ray_ui23422.ipynb?token=8a61dc35207e9ba6c65d8d2704efd0f506dbb7dbe9adc11c



current_dir=/Users/dgrebenyuk/Research/rl-task-planning/gym_kuka_multi_blocks/envs
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Created LogSyncer for /Users/dgrebenyuk/ray_results/2018-10-01_14-17-16xp46ragb -> None
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


VARIABLES collection name is deprecated, please use GLOBAL_VARIABLES instead; VARIABLES will be removed after 2017-03-02.


16.55752967232337
